In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [2]:
#Estableciendo la conexion con la base de datos aniv_cidade_event en MongoDB
db = MongoClient().aniv_cidade_event

In [3]:
#Punto inicial del omnibus 410 Praça Saens Pena
initial_point = (-22.923736, -43.235721)


In [4]:
#Punto final del omnibus 410 Terminal PUC
final_point = (-22.979408, -43.231352)

In [5]:
inverted_initial_point = (-43.235721, -22.923736)

In [6]:
inverted_final_point = (-43.231352, -22.979408)

In [7]:
#Function to compute the bus_ids circulating during a day 20150208 of bus line 438
def bus_id_list ():
    bus_id_20150222_410 = db.dados20150222_line_410.distinct("bus_id")
    return bus_id_20150222_410
    

In [8]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'A72071',
 u'A72006',
 u'A72152',
 u'A72095',
 u'A72065',
 u'A72053',
 u'A72018',
 u'A72012',
 u'A72122',
 u'A72089',
 u'A72118',
 u'A72138',
 u'A72125']

In [9]:
len(busId_list)

13

In [10]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId       
    

In [11]:
array_busId = get_busId_list()
array_busId

['A72071',
 'A72006',
 'A72152',
 'A72095',
 'A72065',
 'A72053',
 'A72018',
 'A72012',
 'A72122',
 'A72089',
 'A72118',
 'A72138',
 'A72125']

In [12]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150222_line_410.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.00052543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [13]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto initial_point(-22.988199999999999, -43.228299999999997) aqui se pasan a la inversa
sorted_near_initial_points = near_points_to_origin ("A72125", inverted_initial_point)
sorted_near_initial_points

,timestamp,latitude,longitude,speed
0,22-02-2015 15:06:25,-22.923893,-43.235806,0.00
1,22-02-2015 17:15:14,-22.923901,-43.236149,0.37
2,22-02-2015 17:15:48,-22.923994,-43.236118,0.19
3,22-02-2015 17:16:23,-22.923950,-43.236099,0.00
4,22-02-2015 17:19:19,-22.923906,-43.236134,0.37
5,22-02-2015 17:20:28,-22.923971,-43.236004,0.37
6,22-02-2015 17:21:05,-22.923958,-43.236004,3.15
7,22-02-2015 19:54:29,-22.924055,-43.235954,0.00
8,22-02-2015 19:55:07,-22.924047,-43.235985,0.00
9,22-02-2015 19:56:17,-22.924047,-43.235985,0.00


In [14]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos a PUC
#Recordar que aqui las coordenadas del punto se pasan a la inversa(long, lat)
sorted_near_final_points = near_points_to_origin ("A72125", inverted_final_point)
sorted_near_final_points

,timestamp,latitude,longitude,speed
0,22-02-2015 16:07:28,-22.979166,-43.231689,13.15
1,22-02-2015 16:08:02,-22.979431,-43.231365,0.74
2,22-02-2015 16:09:12,-22.979204,-43.231316,0.37
3,22-02-2015 16:10:22,-22.979301,-43.231346,0.37
4,22-02-2015 16:11:28,-22.979298,-43.231300,1.30
5,22-02-2015 18:18:03,-22.979166,-43.230923,33.71
6,22-02-2015 18:19:14,-22.979326,-43.231266,0.00
7,22-02-2015 18:20:23,-22.979349,-43.231251,0.37
8,22-02-2015 18:21:04,-22.979359,-43.231281,0.00
9,22-02-2015 18:22:07,-22.979353,-43.231281,0.00


In [15]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [16]:
#pintando los puntos más cerca del punto inicial
fmap=folium.Map(location=[-22.950635, -43.210436], zoom_start=12)

for row in sorted_near_initial_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='violet', num_sides=4, radius=6)

embed_map(fmap)

In [17]:
#pintando los puntos más cerca del punto final (Vila Isabel)
for row in sorted_near_final_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='red', num_sides=4, radius=6)

embed_map(fmap)

In [18]:
#Pintando Circulos de radio de 1km alrededor de los puntos incial y final 
fmap.simple_marker(final_point, popup='Parada de Vila Isabel')
fmap.simple_marker(initial_point, popup='Parada Leblon')

#Punto inicial
fmap.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap.circle_marker(location=initial_point, radius=50,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap)

In [19]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']
initial_points_timestamp

0     22-02-2015 15:06:25
1     22-02-2015 17:15:14
2     22-02-2015 17:15:48
3     22-02-2015 17:16:23
4     22-02-2015 17:19:19
5     22-02-2015 17:20:28
6     22-02-2015 17:21:05
7     22-02-2015 19:54:29
8     22-02-2015 19:55:07
9     22-02-2015 19:56:17
10    22-02-2015 19:57:26
11    22-02-2015 19:58:03
12    22-02-2015 19:59:12
13    22-02-2015 19:59:46
14    22-02-2015 20:04:27
15    22-02-2015 20:05:02
16    22-02-2015 20:06:12
17    22-02-2015 20:07:22
18    22-02-2015 20:08:32
19    22-02-2015 20:09:07
20    22-02-2015 20:10:17
21    22-02-2015 20:11:27
22    22-02-2015 20:12:02
23    22-02-2015 20:13:12
24    22-02-2015 20:14:22
25    22-02-2015 20:15:32
26    22-02-2015 20:16:16
27    22-02-2015 20:16:51
28    22-02-2015 20:17:26
29    22-02-2015 20:20:21
30    22-02-2015 20:21:31
31    22-02-2015 20:22:06
32    22-02-2015 20:23:16
33    22-02-2015 20:26:12
34    22-02-2015 20:26:37
35    22-02-2015 20:28:29
Name: timestamp, dtype: object

In [20]:
final_points_timestamp = sorted_near_final_points['timestamp']
final_points_timestamp

0     22-02-2015 16:07:28
1     22-02-2015 16:08:02
2     22-02-2015 16:09:12
3     22-02-2015 16:10:22
4     22-02-2015 16:11:28
5     22-02-2015 18:18:03
6     22-02-2015 18:19:14
7     22-02-2015 18:20:23
8     22-02-2015 18:21:04
9     22-02-2015 18:22:07
10    22-02-2015 18:22:43
11    22-02-2015 18:25:02
12    22-02-2015 18:26:12
13    22-02-2015 18:27:23
14    22-02-2015 18:28:32
15    22-02-2015 21:25:30
16    22-02-2015 21:26:17
Name: timestamp, dtype: object

In [21]:
#Funcion que retorna el arreglo con las horas de salida del omnibus al punto inicial/final del trayecto
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    return list_departure_hours

In [22]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
leblon_departure_hours = bus_departure_hour(initial_points_timestamp)
leblon_departure_hours

[datetime.datetime(2015, 2, 22, 15, 6, 25),
 datetime.datetime(2015, 2, 22, 17, 21, 5),
 datetime.datetime(2015, 2, 22, 20, 28, 29)]

In [23]:
#Llamada a la funcion bus_departure_hour para el punto Vila Isabel  (devuelve todas las horas en que salio el omnibus de Vila Isabel)
vilaIsabel_departure_hours = bus_departure_hour(final_points_timestamp)
vilaIsabel_departure_hours

[datetime.datetime(2015, 2, 22, 16, 11, 28),
 datetime.datetime(2015, 2, 22, 18, 28, 32),
 datetime.datetime(2015, 2, 22, 21, 26, 17)]

In [24]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    if len(initial_points_timestamp_list) >1:
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    return list_arrive_hours

In [25]:
#Llamada a la funcion bus_arrive_hour para el punto Leblon  (devuelve todas las horas en que llegó el omnibus a Leblon)
arrive_hour_to_initialpoint = bus_arrive_hour(initial_points_timestamp)
arrive_hour_to_initialpoint

[datetime.datetime(2015, 2, 22, 15, 6, 25),
 datetime.datetime(2015, 2, 22, 17, 15, 14),
 datetime.datetime(2015, 2, 22, 19, 54, 29)]

In [26]:
#Llamada a la funcion bus_arrive_hour para el punto Vila Isabel  (devuelve todas las horas en que llegó el omnibus a Vila Isabel)
arrive_hour_to_finalpoint = bus_arrive_hour(final_points_timestamp)
arrive_hour_to_finalpoint

[datetime.datetime(2015, 2, 22, 16, 7, 28),
 datetime.datetime(2015, 2, 22, 18, 18, 3),
 datetime.datetime(2015, 2, 22, 21, 25, 30)]

In [27]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [28]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            direction = name_A + "_to_" + name_B
            departure_hour = departure_array_A[i]
            i = i +1
            arrive_hour = arrive_array_B[j]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
                        
        elif(departure_array_B[j] < departure_array_A[i]):
            direction = name_B + "_to_" + name_A
            departure_hour = departure_array_B[j]
            j = j +1
            arrive_hour = arrive_array_A[i]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  


In [30]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df
        

        #el problema está en sorted_near_initial_points cuando devuelve vacio para algunos bus_id

In [31]:
query = all_travel_table ('410', inverted_initial_point, 'Saens Pena', inverted_final_point, 'PUC', array_busId)
query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,410,A72071,PUC_to_Saens Pena,2015-02-22 07:43:05,2015-02-22 15:28:19,7:45
1,410,A72071,PUC_to_Saens Pena,2015-02-22 12:11:24,2015-02-22 15:28:19,3:16
2,410,A72071,PUC_to_Saens Pena,2015-02-22 14:29:28,2015-02-22 15:28:19,0:58
3,410,A72071,Saens Pena_to_PUC,2015-02-22 15:56:31,2015-02-22 16:56:32,1:00
4,410,A72071,PUC_to_Saens Pena,2015-02-22 17:09:07,2015-02-22 18:23:09,1:14
5,410,A72071,Saens Pena_to_PUC,2015-02-22 18:29:32,2015-02-22 19:27:29,0:57
6,410,A72071,PUC_to_Saens Pena,2015-02-22 19:35:09,2015-02-22 20:40:16,1:05
7,410,A72065,Saens Pena_to_PUC,2015-02-22 05:53:45,2015-02-22 11:50:02,5:56
8,410,A72065,Saens Pena_to_PUC,2015-02-22 08:42:23,2015-02-22 11:50:02,3:07
9,410,A72065,PUC_to_Saens Pena,2015-02-22 11:57:06,2015-02-22 15:23:32,3:26
